In [2]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


/workspace


In [3]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [4]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


In [5]:
date :str = "2022年8月26日"


input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

targetFunctionNames: list[str] = [
    "VECSET",
    "MPI_Wait()",
    "MPI_Send()",
    "SPRNVC",
    "RANDLC",
    "MPI_Irecv()",
    "ICNVRT",
]

function_names: list[str] = list(set(input_rawDF_cg["Name"].tolist()))

input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})


/workspace/lib/lab_lib.py:8781: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/workspace/lib/lab_lib.py:8781: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [6]:
result_series_list: list[pd.DataFrame] = []


list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    # "modelSquareRootOfProcess",
    # "modelSquareRootTimesOtherElems",
    # "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_before = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

resultDF_before

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,functionName,最低値,最適モデル
0,1.056736e-13,40.738362,24.184347,59.575073,95.488585,40.738362,24.184347,3.109564e-14,24.184348,3.032598e-14,40.738362,14.024686,1.606494e-12,19.369661,25.924923,SPRNVC,3.032598e-14,modelLogAndLin
1,3.425747e+01,52.562966,45.284269,48.827706,381.319302,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Send(),1.193837e+01,modelLinearSumOfElementCombinations
2,2.550716e+01,63.602569,43.918421,68.457716,68.703722,63.602561,43.918423,2.550717e+01,43.918430,2.550716e+01,63.602559,13.725277,1.353524e+01,26.547929,36.391398,RANDLC,1.353524e+01,modelLinearSumOfElementCombinations
3,2.550709e+01,63.602611,43.918416,68.457739,68.707184,63.602603,43.918418,2.550710e+01,43.918426,2.550709e+01,63.602600,13.725285,1.353524e+01,26.548026,36.391518,ICNVRT,1.353524e+01,modelLinearSumOfElementCombinations
4,3.425747e+01,52.562966,45.284269,48.827706,381.319302,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Wait(),1.193837e+01,modelLinearSumOfElementCombinations
5,3.425747e+01,52.562966,45.284269,48.827706,381.319302,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Irecv(),1.193837e+01,modelLinearSumOfElementCombinations
6,1.056736e-13,40.738362,24.184347,59.575073,95.488585,40.738362,24.184347,3.109564e-14,24.184348,3.032598e-14,40.738362,14.024686,1.606494e-12,19.369661,25.924923,VECSET,3.032598e-14,modelLogAndLin


In [7]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_before: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [8]:
print(resultDF_before.loc[:, ["functionName", "最低値", "最適モデル"]].sort_values("functionName").rename(columns={"functionName":"関数名", "最低値":"MAPE"}).style.to_latex())

print(f"retNum_cg_before = {retNum_cg_before}")

\begin{tabular}{llrl}
 & 関数名 & MAPE & 最適モデル \\
3 & ICNVRT & 13.535235 & modelLinearSumOfElementCombinations \\
5 & MPI_Irecv() & 11.938374 & modelLinearSumOfElementCombinations \\
1 & MPI_Send() & 11.938374 & modelLinearSumOfElementCombinations \\
4 & MPI_Wait() & 11.938374 & modelLinearSumOfElementCombinations \\
2 & RANDLC & 13.535239 & modelLinearSumOfElementCombinations \\
0 & SPRNVC & 0.000000 & modelLogAndLin \\
6 & VECSET & 0.000000 & modelLogAndLin \\
\end{tabular}

retNum_cg_before = 7.827486056619932


In [9]:
result_series_list: list[pd.DataFrame] = []


list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_after = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

resultDF_after

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,...,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,modelSquareRootOfProcess,modelSquareRootTimesOtherElems,modelObeyOneParameter,functionName,最低値,最適モデル
0,1.056736e-13,40.738362,24.184347,59.575073,95.488585,40.738362,24.184347,3.109564e-14,24.184348,3.032598e-14,...,14.024686,1.606494e-12,19.369661,25.924923,3.326790e-13,79.407704,0.000000,SPRNVC,0.000000,modelObeyOneParameter
1,3.425747e+01,52.562966,45.284269,48.827706,381.319302,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Send(),11.938374,modelLinearSumOfElementCombinations
2,2.550716e+01,63.602569,43.918421,68.457716,68.703722,63.602561,43.918423,2.550717e+01,43.918430,2.550716e+01,...,13.725277,1.353524e+01,26.547929,36.391398,2.550716e+01,110.789118,25.507161,RANDLC,13.535239,modelLinearSumOfElementCombinations
3,2.550709e+01,63.602611,43.918416,68.457739,68.707184,63.602603,43.918418,2.550710e+01,43.918426,2.550709e+01,...,13.725285,1.353524e+01,26.548026,36.391518,2.550709e+01,110.789284,25.507090,ICNVRT,13.535235,modelLinearSumOfElementCombinations
4,3.425747e+01,52.562966,45.284269,48.827706,381.319302,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Wait(),11.938374,modelLinearSumOfElementCombinations
5,3.425747e+01,52.562966,45.284269,48.827706,381.319302,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Irecv(),11.938374,modelLinearSumOfElementCombinations
6,1.056736e-13,40.738362,24.184347,59.575073,95.488585,40.738362,24.184347,3.109564e-14,24.184348,3.032598e-14,...,14.024686,1.606494e-12,19.369661,25.924923,3.326790e-13,79.407704,0.000000,VECSET,0.000000,modelObeyOneParameter


In [10]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_after: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [11]:
print(resultDF_after.loc[:, ["functionName", "最低値", "最適モデル"]].sort_values("functionName").rename(columns={"functionName":"関数名", "最低値":"MAPE"}).style.to_latex())

print(f"retNum_cg_after = {retNum_cg_after}")

\begin{tabular}{llrl}
 & 関数名 & MAPE & 最適モデル \\
3 & ICNVRT & 13.535235 & modelLinearSumOfElementCombinations \\
5 & MPI_Irecv() & 11.938374 & modelLinearSumOfElementCombinations \\
1 & MPI_Send() & 11.938374 & modelLinearSumOfElementCombinations \\
4 & MPI_Wait() & 11.938374 & modelLinearSumOfElementCombinations \\
2 & RANDLC & 13.535239 & modelLinearSumOfElementCombinations \\
0 & SPRNVC & 0.000000 & modelObeyOneParameter \\
6 & VECSET & 0.000000 & modelObeyOneParameter \\
\end{tabular}

retNum_cg_after = 7.827486056619932


In [12]:
%reset

Nothing done.


***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___